In [13]:
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Sequential

In [14]:
with open('../../assets/annotations.json', 'r') as file:
    annotations = json.load(file)

In [15]:
labels = {'standing': 0, 'takedown': 1, 'open_guard': 2, 'half_guard': 3, 'closed_guard': 4, '5050_guard': 5, 'side_control': 6, 'mount': 7, 'back': 8, 'turtle': 9}
num_labels = len(labels)
num_keypoints = 17
num_players = 2

In [16]:
import numpy as np
from random import shuffle
import math

def translate_keypoints(keypoints, dx, dy):
    return [(x + dx, y + dy) for x, y in keypoints]

def scale_keypoints(keypoints, scale):
    return [(x * scale, y * scale) for x, y in keypoints]

def rotate_keypoints(keypoints, angle, center):
    angle_rad = math.radians(angle)
    cos_theta, sin_theta = math.cos(angle_rad), math.sin(angle_rad)
    cx, cy = center
    return [
        (
            cos_theta * (x - cx) - sin_theta * (y - cy) + cx,
            sin_theta * (x - cx) + cos_theta * (y - cy) + cy
        )
        for x, y in keypoints
    ]

def flip_keypoints(keypoints, image_width):
    return [(image_width - x, y) for x, y in keypoints]

def add_noise(keypoints, noise_level):
    return [(x + np.random.normal(0, noise_level), y + np.random.normal(0, noise_level)) for x, y in keypoints]

In [17]:
from random import shuffle

data = []

try:
    for annotation in annotations:
        label = annotation['position']
        if label[-1].isdigit():
            label = label[:-1]
        
        if annotation.get('pose1'):
            keypoints1 = [keypoint[:2] for keypoint in annotation['pose1']]
        else:
            keypoints1 = [[0] * 2] * num_keypoints
            
        if annotation.get('pose2'):
            keypoints2 = [keypoint[:2] for keypoint in annotation['pose2']]
        else:
            keypoints2 = [[0] * 2] * num_keypoints

        # keypoints originais
        original_keypoints = keypoints1 + keypoints2
        keypoints_array = np.array(original_keypoints).astype(np.float32).reshape(num_keypoints * num_players * 2)
        max_x = max(keypoints_array)
        normalized_keypoints = keypoints_array / max_x
        data.append((normalized_keypoints, labels[label]))
        
        # keypoints com poses invertidas 
        inverted_keypoints = keypoints2 + keypoints1
        inverted_keypoints_array = np.array(inverted_keypoints).astype(np.float32).reshape(num_keypoints * num_players * 2)
        max_x = max(inverted_keypoints_array)
        normalized_inverted_keypoints = inverted_keypoints_array / max_x
        data.append((normalized_inverted_keypoints, labels[label]))

        for keypoints in [original_keypoints, inverted_keypoints]:
            augmentations = []
            
            # 1. Translation
            dx, dy = np.random.randint(-20, 20), np.random.randint(-20, 20)
            translated_keypoints = translate_keypoints(original_keypoints, dx, dy)
            augmentations.append(translated_keypoints)

            # 2. Scaling
            scale_factor = np.random.uniform(0.8, 1.2)
            scaled_keypoints = scale_keypoints(original_keypoints, scale_factor)
            augmentations.append(scaled_keypoints)

            # 3. Rotation
            angle = np.random.uniform(-30, 30)
            xs = [x for x, y in original_keypoints]
            ys = [y for x, y in original_keypoints]
            center = (np.mean(xs), np.mean(ys))
            rotated_keypoints = rotate_keypoints(original_keypoints, angle, center)
            augmentations.append(rotated_keypoints)

            # 4. Horizontal Flip
            image_width = max(xs)  
            flipped_keypoints = flip_keypoints(original_keypoints, image_width)
            augmentations.append(flipped_keypoints)

            # 5. Add Noise
            noisy_keypoints = add_noise(original_keypoints, noise_level=2)
            augmentations.append(noisy_keypoints)

            for aug_keypoints in augmentations:
                aug_keypoints_array = np.array(aug_keypoints).astype(np.float32).reshape(num_keypoints * num_players * 2)
                max_x = max(aug_keypoints_array)
                normalized_aug_keypoints = aug_keypoints_array / max_x
                data.append((normalized_aug_keypoints, labels[label]))

except Exception as e:
    print("Error:", e)

keypoints_list = []
labels_list = []

shuffle(data)

for keypoints, label in data:
    keypoints_list.append(keypoints)
    labels_list.append(label)

labels_list = np.array(labels_list)

KeyboardInterrupt: 

In [9]:
TRAIN_PERCENTAGE = 0.8

train_keypoints = keypoints_list[:int(len(keypoints_list) * TRAIN_PERCENTAGE)]
train_labels = labels_list[:int(len(labels_list) * TRAIN_PERCENTAGE)]

test_keypoints = keypoints_list[int(len(keypoints_list) * TRAIN_PERCENTAGE):]
test_labels = labels_list[int(len(labels_list) * TRAIN_PERCENTAGE):]

train_keypoints = np.array(train_keypoints)
train_labels = tf.keras.utils.to_categorical(train_labels)

test_keypoints = np.array(test_keypoints)
test_labels = tf.keras.utils.to_categorical(test_labels)

print("Train: ", len(train_keypoints))
print("Test: ", len(test_keypoints))

print("Label example: ", train_labels[65])
print("Keypoint example", train_keypoints[65])

Train:  1154678
Test:  288670
Label example:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Keypoint example [0.46629515 0.2835518  0.46101752 0.2722335  0.45167482 0.27768376
 0.4408722  0.27735952 0.41895014 0.2947803  0.47754714 0.30245635
 0.39398727 0.35766214 0.5361657  0.29786906 0.3822555  0.44908518
 0.6006186  0.29692796 0.40021443 0.5270957  0.4922452  0.42654905
 0.46207696 0.4825509  0.56597906 0.33319393 0.5046792  0.434746
 0.64070773 0.42629018 0.6005842  0.4570997  0.88205075 0.20180856
 0.87821513 0.19232537 0.8910183  0.19565324 0.86784285 0.18635692
 0.8994012  0.19425465 0.82073575 0.16865113 0.9165882  0.18104266
 0.7268421  0.16230437 0.9504453  0.26144308 0.7183461  0.23151894
 1.         0.3414818  0.72160345 0.1800137  0.7908244  0.19203483
 0.7661923  0.28955936 0.8876098  0.30109596 0.77565306 0.39810595
 0.76628655 0.35276145]


In [10]:
model = Sequential([
    layers.Dense(512, activation='relu', input_shape=(num_keypoints * 2 * num_players,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_labels, activation='softmax')
])

model.summary()

/home/waizbart/Documents/projects/bjj_ia/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1730131387.213517   39256 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-28 13:03:07.245876: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │        35,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,898 (429.29 KB)

 Trainable params: 109,898 (429.29 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
checkpoint_path = "weights.best.keras"

checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

history = model.fit(train_keypoints, train_labels,
                    epochs=100,
                    batch_size=16,
                    validation_data=(test_keypoints, test_labels),
                    callbacks=[checkpoint, earlystopping])

Epoch 1/100


2024-10-28 13:03:40.047488: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 314072416 exceeds 10% of free system memory.


72102/72168 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.7053 - loss: 0.8337
Epoch 1: val_accuracy improved from -inf to 0.84901, saving model to weights.best.keras
72168/72168 ━━━━━━━━━━━━━━━━━━━━ 62s 846us/step - accuracy: 0.7054 - loss: 0.8335 - val_accuracy: 0.8490 - val_loss: 0.4299
Epoch 2/100
72129/72168 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.8561 - loss: 0.4099
Epoch 2: val_accuracy improved from 0.84901 to 0.88691, saving model to weights.best.keras
72168/72168 ━━━━━━━━━━━━━━━━━━━━ 65s 901us/step - accuracy: 0.8561 - loss: 0.4099 - val_accuracy: 0.8869 - val_loss: 0.3303
Epoch 3/100
72163/72168 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.8838 - loss: 0.3358
Epoch 3: val_accuracy improved from 0.88691 to 0.90305, saving model to weights.best.keras
72168/72168 ━━━━━━━━━━━━━━━━━━━━ 62s 853us/step - accuracy: 0.8838 - loss: 0.3358 - val_accuracy: 0.9030 - val_loss: 0.2858
Epoch 4/100
72118/72168 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.8994 - loss: 0.293

In [ ]:
model.evaluate(test_keypoints, test_labels)

In [ ]:
import matplotlib.pyplot as plt

# Plot Utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
model = tf.keras.models.load_model(checkpoint_path)

keypoints = np.array(annotations[0]['pose2'])

keypoints = [[0] * 2] * num_keypoints
keypoints += [keypoint[:2] for keypoint in keypoints]

keypoints = np.array(keypoints).astype(np.float32).reshape(num_keypoints * num_players * 2)

print(keypoints.shape)

max_x = max(keypoints)
normalized_keypoints = keypoints / max_x

prediction = model.predict(normalized_keypoints.reshape(1, num_keypoints * 2 * num_players))

print(prediction)

max_index = np.argmax(prediction[0])

for label, index in labels.items():
    if index == max_index:
        print(label)
        break

In [ ]:
# Calculate accuracy for each label

test_dict = {}

n = len(test_keypoints)

print(n)

for i in range(0, n):
    test = np.array([test_keypoints[i]])

    prediction = model.predict(test)
    
    correct_label = np.argmax(test_labels[i])
    predicted_label = np.argmax(prediction)
    
    label_name = list(labels.keys())[predicted_label]
    
    if predicted_label == correct_label:
        if label_name in test_dict:
            test_dict[label_name]["correct"] += 1
        else:
            test_dict[label_name] = {"correct": 1, "incorrect": 0}
    else:
        if label_name in test_dict:
            test_dict[label_name]["incorrect"] += 1
        else:
            test_dict[label_name] = {"correct": 0, "incorrect": 1}
    
for key in test_dict:
    correct = test_dict[key]["correct"]
    incorrect = test_dict[key]["incorrect"]
    
    print(f"Accuracy for {key}: {correct / (correct + incorrect)}")
